# CSCI 3202:  Forward-Backward Algorithm (Filtering and Smoothing)

### The set-up

There is a **hidden Markov model**, whose states $X_t$ cannot be observed directly.  We can, however, make observations of $E_t$, which is a kind of **proxy** for the process of interest $X_t$. We want to use our observations $E_t$ to make inferences regarding the unobservable (hidden!) process $X_t$.

We are given the following:
* Markov model transition probabilities, $P(X_{t+1} \mid X_t)$ (how does the process of interest evolve over time?)
* sensor model probabilities, $P(E_t \mid X_t)$ (how do our measurements depend on the actual process?)
* prior distribution, $P(X_0)$ (how did the process start?)

Suppose we have a whole bunch of evidence, from time $k=1$ all the way up through time $k=t$ (consider $t$ to be the "present", or at least the time at which our observing system stops making measurements).

Recall that `Forward` filtering updates our estimate of the probability distribution for $X_t$ in light of *all* of the evidence up to time $t$, $E_{1:t}$, as

$$P(X_t \mid E_{1:t}) = f_{1:t} = \alpha ~ \texttt{forward}(f_{1:t-1}, E_t)$$
 
The `Backward` function is a necessary step to update our estimate of the probability distribution for $X_k$ (some $k$ in $[0,t]$) in light of *all* of the evidence that occurred after $k$, $E_{k+1:t}$, as

$$P(E_{k+1:t} \mid X_k) = b_{k+1:t} = \texttt{backward}(b_{k+2:t}, E_{k+1})$$

where the specific functional forms of these recurrences we will dissect below.

<br>

### Forward algorithm / filtering

This is a **single** iteration of the `forward` algorithm.  We take a new observation $E_{t}$, and the old probability distribution of $X_{t-1}$ given the previous evidence set $E_{1:t-1}$, $P(X_{t-1} \mid E_{1:t-1}$, to update this to the new probability distribution of $X_t$ given all of the evidence $E_{1:t}$, $P(X_t \mid E_{1:t})$.

This update is described by this equation:

$\begin{align*}
  P(X_t \mid E_{1:t}) = f_{1:t} &= \texttt{forward}(f_{1:t-1}, E_t) \\
                     &= \alpha ~ P(E_t \mid X_t) \sum_{x_{t-1}} P(X_t \mid X_{t-1}) \underbrace{P(X_{t-1} \mid E_{1:t-1})}_{f_{1:t-1}}
 \end{align*}$

<br>

#### Example:  (from the review session)

* Markov model: $P(X_{k+1}=T \mid X_k=T) = 0.7$ and $P(X_{k+1}=T \mid X_k=F) = 0.5$
* Sensor model: $P(E_k=T \mid X_k=T) = 0.9$ and $P(E_k=T \mid X_k=F) = 0.2$
* Prior: $P(X_0=T) = 0.6$ and $P(X_0=F) = 0.4$

Suppose we have $E_1 = T$ and $E_2 = T$ as our data.

Then the filtered estimate of $P(X_1 \mid E_1=T)$ is:

$$P(X_1 \mid E_1=T) = \alpha P(E_1=T \mid X_1) \sum_{x_0} P(X_1 \mid x_0) P(x_0 \mid E_{1:0}) = \alpha P(E_1=T \mid X_1) \sum_{x_0} P(X_1 \mid x_0) f_{1:0}$$

but $E_{1:0}$ does not really make any sense.  So we start things off with the prior distribution, $P(X_0)$, taking the place of $f_{1:0}$ (because there isn't really any evidence to incorporate there yet).

So we have:

$\begin{align*}
  f_{1:1} = P(X_1 \mid E_{1:1}) &= \alpha P(E_1=T \mid X_1 \sum_{x_0} P(X_1 \mid x_0) P(x_0) \\
    &= \alpha \left( \begin{array}{c} P(E_1=T \mid X_1=T) \\ P(E_1=T \mid X_1=F) \end{array}\right) \times \left( \begin{array}{c} \sum_{x_0} P(X_1=T \mid x_0)P(x_0) \\ \sum_{x_0} P(X_1=F \mid x_0)P(x_0) \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} P(E_1=T \mid X_1=T) \\ P(E_1=T \mid X_1=F) \end{array}\right) \times \left( \begin{array}{c} P(X_1=T \mid X_0=T)P(X_0=T)+P(X_1=T \mid X_0=F)P(X_0=F) \\ P(X_1=F \mid X_0=T)P(X_0=T)+P(X_1=F \mid X_0=F)P(X_0=F) \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} 0.9 \\ 0.2 \end{array}\right) \times \left( \begin{array}{c} 0.7\cdot 0.6 + 0.5\cdot 0.4 \\ 0.3\cdot 0.6 + 0.5\cdot 0.4 \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} 0.9 \\ 0.2 \end{array}\right) \times \left( \begin{array}{c} 0.62 \\ 0.38 \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} 0.558 \\ 0.076 \end{array}\right) \\
\end{align*}$

Normalizing so that the elements of $f_{1:1}$ sum to 1, we find:

$$f_{1:1} = P(X_1 \mid E_{1:1}) = \left( \begin{array}{c} 0.558/(0.558+0.076) \\ 0.076/(0.558+0.076) \end{array}\right) = \left( \begin{array}{c} 0.88 \\ 0.12 \end{array}\right)$$

The operation of the `forward` function here would be to take in $f_{1:0} = P(X_0)$ (an array with a probability values for each possible value of $X_0$) and the single piece of evidence $E_1$ and return the probability distribution $P(X_1 \mid E_{1:1})$, an array with probability values for each possible value of $X_1$.

Then, to get $f_{1:2} = P(X_2 \mid E_{1:2})$, we would send into `forward` $f_{1:1}$ (which we just calculated) and the new evidence $E_2$:

$\begin{align*}
  f_{1:2} = P(X_2 \mid E_{1:2}) &= \alpha P(E_2=T \mid X_2 \sum_{x_1} P(X_2 \mid x_1) P(x_1 \mid E_{1:1}) \\
    &= \alpha \left( \begin{array}{c} P(E_2=T \mid X_2=T) \\ P(E_2=T \mid X_2=F) \end{array}\right) \times \left( \begin{array}{c} \sum_{x_1} P(X_2=T \mid x_1)P(x_1 \mid E_{1:1}) \\ \sum_{x_1} P(X_2=F \mid x_1)P(x_1 \mid E_{1:1}) \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} P(E_2=T \mid X_2=T) \\ P(E_2=T \mid X_2=F) \end{array}\right) \times \left( \begin{array}{c} P(X_2=T \mid X_1=T) \underbrace{P(X_1=T \mid E_{1:1})}_{\text{from }f_{1:1}}+P(X_2=T \mid X_1=F) \underbrace{P(X_1=F \mid E_{1:1})}_{\text{from }f_{1:1}} \\ P(X_2=F \mid X_1=T) \underbrace{P(X_1=T \mid E_{1:1})}_{\text{from }f_{1:1}}+P(X_2=F \mid X_1=F) \underbrace{P(X_1=F \mid E_{1:1})}_{\text{from }f_{1:1}} \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} 0.9 \\ 0.2 \end{array}\right) \times \left( \begin{array}{c} 0.7\cdot 0.88 + 0.5\cdot 0.12 \\ 0.3\cdot 0.88 + 0.5\cdot 0.12 \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} 0.9 \\ 0.2 \end{array}\right) \times \left( \begin{array}{c} 0.676 \\ 0.324 \end{array}\right) \\
    &= \alpha \left( \begin{array}{c} 0.6084 \\ 0.0648 \end{array}\right) \\
    &= \left( \begin{array}{c} 0.90 \\ 0.10 \end{array}\right) ~~~~~ \text{(normalizing with $\alpha=1/(0.6084+0.0648)$)}\\
\end{align*}$

Now, one option for getting all the different entries in $f_{1:2}$ would be to cast this as a matrix-vector multiplication. But linear algebra is not a prerequisite for this class, so we have shied away from doing too much of that. The other option is to loop over each possible value of $X_2$ and fill in $f_{1:2}$ as we go. And the sum over $X_1$ within the calculation of $f_{1:2}$ can be done with a second, nested, "for" loop.

Below is some pseudocode for the `forward` function.

In [ ]:
def forward(f1, e, [other arguments?]):
    '''
    Returns the probability distribution of X at the current time
    step t, in light of all the evidence up through and including
    the newest data point at time step t, E_t.
    f1 -- probability distribution of X at previous time step, given
          the evidence up until that time step, P(X_{t-1} | E_{1:t-1})
    e  -- the new evidence point, E_t
    xv -- all of the possible values for our random variable, X
          (For example, xv = [True, False] for a Boolean, or
           xv = [0, 1, 2, 3] for X representing the position of an object 
           that is in exactly one of 4 bins... let's just say)
    '''
    
    # initialize output f as an array to store a probability
    # for each possible value of X at time t
    
    # Loop 1 is over X_t, filling it in with unnormalized probabilities
    # (s'pose Xi is the value of X_t that we are currently filling
    #  in the probability of)

        # calculate the first term, P(e | Xi)
        # in each of Loop 1, X_t = Xi, and evidence E_t = e is input

        # calculate the [sum], using Loop 2 over X_{t-1} possible values
        # (s'pose Xi1 is the value of X_{t-1} that Loop 2 is currently
        #  working on)

            # first term in each part of the sum is P(X_t | X_{t-1})
            # --> X_t is given by Loop 1:  Xi
            # --> X_{t-1} given by Loop 2:  Xi1
            # --> The relevant probability must be grabbed from the Markov model
            
            # second term in each part of the sum is P(X_{t-1} | E_{1:t-1})
            # --> this is the input, f1
            # --> f1 = P(X_{t-1} | E_{1:t-1}), so it is an array with different
            #     probabilities for each different X_{t-1}.  
            #     So Loop 2 gives the proper element to reference here.
            
            # add to our sum P(X_{t+1} | X_t)*P(X_{t-1} | E_{1:t-1})

        # f[Xi] = P(e | Xi) * [sum]
        
    # normalize the output f
    
    return f

### Backward algorithm, and smoothing

In order to obtain **smoothed** estimates of a state $X_1$ in light of all the evidence, say, $E_{1:2}$, we need to run the `backward` algorithm, as well as the `forward` one.  The following is a **single iteration** of `backward`, to calculate the probability of seeing the evidence we would *later* see, given a particular possible state of the process at time $k$.

$\begin{align*}
  P(E_{k+1:t} \mid X_k) = b_{k+1:t} &= \texttt{backward}(b_{k+2:t}, E_{k+1}) \\
                     &= \sum_{x_{k+1}} P(E_{k+1} \mid X_{k+1}) P(X_{k+1} \mid X_{k}) \underbrace{P(E_{k+2:t} \mid X_{k+1})}_{b_{k+2:t}}
 \end{align*}$

Why do we want these $b$'s anyway?  Well, **smoothing** updates our previous (filtered) estimates of $P(X_k \mid E_{1:k})$ by incorporating *all* of the data, as $P(X_k \mid E_{1:t})$ (where we are assuming that $k < t$). Our smoothed estimates we saw are given by:

$P(X_k \mid E_{1:t}) = \alpha ~ f_{1:k} \times b_{k+1:t}$
 
<br>

#### Example, continued:  (from the review session)

... Suppose we have $E_1 = T$ and $E_2 = T$ as our data, and we have the results for $f_{1:1}$ and $f_{1:2}$ from above.

Then the smoothed estimate of $P(X_1 \mid E_{1:2})$ is:

$$P(X_1 \mid E_{1:2}) = \alpha ~ f_{1:1} \times b_{2:2}$$

We already have $f_{1:1}$, so we need to calculate $b_{2:2}$ using our `backward` algorithm.

$\begin{align*}
  b_{2:2} = P(E_{2:2} \mid X_1) &= \sum_{x_2} P(E_2=T \mid x_2) P(x_2 \mid X_1) \underbrace{P(E_{3:2} \mid X_2)}_{=1} \\
    &= \left( \begin{array}{c} \sum_{x_2} P(E_2=T \mid x_2) P(x_2 \mid X_1=T) \\  \sum_{x_2} P(E_2=T \mid x_2) P(x_2 \mid X_1=F) \end{array} \right) \\
    &= \left( \begin{array}{c} P(E_2=T \mid X_2=T) P(X_2=T \mid X_1=T) + P(E_2=T \mid X_2=F) P(X_2=F \mid X_1=T) \\  P(E_2=T \mid X_2=T) P(X_2=T \mid X_1=F) + P(E_2=T \mid X_2=F) P(X_2=F \mid X_1=F) \end{array} \right) \\
    &= \left( \begin{array}{c} 0.9 \cdot 0.7 + 0.2\cdot 0.3 \\  0.9 \cdot 0.5 + 0.2 \cdot 0.5 \end{array} \right) \\
    &= \left( \begin{array}{c} 0.69 \\ 0.55 \end{array} \right) \\
\end{align*}$

(Note that a couple arithmetic errors in the Final Review session slides are fixed here.)

So, to get our smoothed estimate $P(X_1 \mid E_{1:2})$, we calculate:

$\begin{align*}
 P(X_1 \mid E_{1:2}) &= \alpha ~ f_{1:1} \times b_{2:2} \\
  &= \alpha ~ \left( \begin{array}{c} 0.88 \\ 0.12 \end{array} \right) \times \left( \begin{array}{c} 0.69 \\ 0.55 \end{array} \right) \\
  &= \alpha ~ \left( \begin{array}{c} 0.6072 \\ 0.066 \end{array} \right) \\
  &= \left( \begin{array}{c} 0.989 \\ 0.011 \end{array} \right) ~~~~ \text{normalizing with $\alpha=1/(0.6072+0.066)$}\\
\end{align*}$

Now, for calculating $b_{k+1:t}$ based on $b_{k+2:t}$ and $E_{k+1}$, we could again use a matrix-vector multiplication.  Or we could do something quite similar to the `forward` algorithm and construct one loop to fill in our output array $b_{k+1:t}$, and a second nested loop to do the sum over $X_{k+1}$. We start by calculating $b_{t:t}$, where $b_{t+1:t}$ is simply defined to be 1. Below is some pseudocode for the `backward` algorithm.

In [ ]:
def backward(b1, e, [other arguments?]):
    '''
    Returns the probabilities of the future evidence E_{k+1:t}, supposing
    a particular value for the state of our Markov process at time k, X_k.
    Note that the output is NOT a probability distribution, so it will not
    in general sum to 1.
    b1 -- probabilities of E_{k+2:t} given X_{k+1}; an array with a value
          for each possible value of X_{k+1}
    e  -- the new evidence point, E_{k+1}
    xv -- all of the possible values for our random variable, X
    '''
    
    # initialize output b as an array to store a probability
    # for each possible value of X at time k
    
    # Loop 1 is over X_k, filling it in with unnormalized probabilities
    # (s'pose Xi is the value of X_k that we are currently filling in
    #  P(E_{k+1:t} | X_k) for)

        # calculate the [sum], using Loop 2 over X_{k+1} possible values
        # (s'pose Xi1 is the value of X_{k+1} that Loop 2 is currently
        #  working on)

            # first term in each part of the sum is P(E_{k+1} | X_{k+1})
            # --> E_{k+1} is the evidence input
            # --> X_{k+1} is given by where we are in Loop 2 (Xi1)
            # --> Relevant probability must be grabbed from the sensor model.

            # second term in each part of the sum is P(X_{k+1} | X_k)
            # --> X_k is given by Loop 1:  Xi
            # --> X_{k+1} given by Loop 2:  Xi1
            # --> The relevant probability must be grabbed from the Markov model
            
            # third term in each part of the sum is P(E_{k+2:t} | X_{k+1})
            # --> this is the input, b1
            # --> b1 is an array corresponding to different values for X_{k+1},
            #     so Loop 2 gives the proper element to reference here.
    
    return b

### Forward-backward algorithm

Alright, so we've got `forward` and `backward`, what the heck is the `forward-backward` and how do we get it from the functions we defined above?

`forward-backward` algorithm produces the smoothed estimates by doing two steps:
1. iterate forward from $k=1$ to $k=t$ and calculate $f_{1:k}$ for each $k$
2. iterate backward from $k=t$ to $k=1$ and calculate $b_{k+1:t}$ for each $k$

During the backward iteration, you can compute the smoothed estimates as: 

$$s_k = P(X_k \mid E_{1:t}) = \alpha ~ f_{1:k} \times b_{k+1:t},$$

where you will need to find the normalizing constant $\alpha$ such that $s_k$ sums to 1.  Note that $s_k$ is a probability distribution over $X_k$.

You will need to save all of the $f_{1:k}$ distributions for each $k$ because you'll need them for the smoothing.  But you do not need to save all of the $b_{k+1:t}$ arrays, since during the backwards iteration you can compute $b$, then use it to compute $s_k$ which is the thing you want to save, then overwrite $b$ on the next iteration. $s_k$ and $f_{1:k}$ are the only things you really care about, the smoothed and filtered estimates, respectively.